In [1]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

In [5]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df["Launch Site"].value_counts()
df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [14]:
app = JupyterDash(__name__)

# Build dash app layout
app.layout = html.Div(children=[ 
    html.H1("Space X Launch Records Dashboard",
        style={'textAlign': 'center', 'color': '#503D36','font-size': 35}),
    html.Div([ 
        dcc.Dropdown(
            id='site_dropdown',
                        options=[
                             {'label': 'All Sites', 'value': 'ALL'},
                             {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                             {'label': 'KSC LC-39A', 'value': 'VAFB SLC-4E'},
                             {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                ],
                value='ALL',
                placeholder="Select a Launch Site here",
                searchable=True
                ),
        
        html.Div(dcc.Graph(id='plot1')),
        html.Br(),
        
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(id='payload_slider',
                        min=0, max=10000, step=1000,
                value=[min(df["Payload Mass (kg)"]),
                       max(df["Payload Mass (kg)"])],
                tooltip={"placement": "bottom", "always_visible": True}),
        
        html.Div(dcc.Graph(id='plot2')),
        ])
    ])
    
@app.callback(Output(component_id='plot1', component_property='figure'),
              Input(component_id='site_dropdown', component_property='value'))
              
def get_pie_chart(value):
    if value == 'ALL':
        fig = px.pie(values=df["Launch Site"].value_counts().values,
                        names=df['Launch Site'].value_counts().index,
        title='Total Sucess Launches by Site')
        return fig
    else:
        fdf = df[df["Launch Site"] == value]
        fig = px.pie(values=fdf['class'].value_counts().values, 
        names=fdf['class'].value_counts().index,
        title=(f'Total Succes Launces for site {value}'))
        return fig

@app.callback(Output(component_id='plot2', component_property='figure'),
    Input(component_id='site_dropdown', component_property='value'),
    Input(component_id='payload_slider', component_property='value'))
             
def get_scatter_chart(value,payload_slider):
    if value == 'ALL':
        fig2 = px.scatter(df,x ="Payload Mass (kg)", y = "class",
        color= "Booster Version Category",
        range_x=[payload_slider[0],payload_slider[1]],                  
        title='Correlation between Payload and Sucess for all Sites')
        return fig2
    else:
        fdf = df[df["Launch Site"] == value]
        fig2 = px.scatter(fdf,x = "Payload Mass (kg)", y = "class",
        range_x=[payload_slider[0],payload_slider[1]],  
        color= "Booster Version Category",
        title=(f'Correlation between Payload and Sucess for site {value}'))
        return fig2
    
def update_fig_axis_range(fig2,payload_slider):
    
    fig2.update_xaxes(autorange=False,range=payload_slider)
    return fig2
    

app.run_server( host = '127.0.0.1', port= 8055)
               

Dash app running on http://127.0.0.1:8055/
